# Synthetic Transaction Dataset

This notebook generates a realistic synthetic transaction dataset (India context) and injects a variety
of fraud patterns (card cloning, account takeover, merchant collusion, velocity probes, etc.).


## Final combined pattern list (implemented / referenced)


Implemented patterns:
- Log-normal amounts by merchant category (realistic skew)
- Geospatial centroids for customers and merchants + Haversine distance
- Temporal seasonality (hour peaks) and weekly modulation
- Sticky customer category preferences
- Card cloning: impossible travel & concurrent sessions
- Account takeover (ATO): bursty high-value sequences, category shift
- Merchant collusion: clique-like behavior and structured amounts near ₹50,000
- Velocity probes and small-value testing
- Post-generation risk-score adjustment to hit target fraud rate (2%)


In [ ]:
# Standard imports and global configuration
import numpy as np
import pandas as pd
import math
import random
from datetime import datetime, timedelta, timezone
import os

# Reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Output path
OUT_DIR = './'
OUT_CSV = os.path.join(OUT_DIR, 'transactions.csv')

# Dataset size & characteristics
N_ROWS = 100_000
N_CUSTOMERS = 5000
N_MERCHANTS = 500
TARGET_FRAUD_RATE = 0.02  

START_DATE = datetime(2025, 1, 1)
END_DATE = datetime(2025, 3, 31)

# Merchant categories & simple regional bounding box (India)
MERCHANT_CATEGORIES = ['grocery', 'electronics', 'gas', 'restaurant', 'retail', 'jewelry', 'luxury_goods']
LAT_MIN, LAT_MAX = 6.5, 37.8
LON_MIN, LON_MAX = 68.0, 97.5

print('Notebook configured — will write to', OUT_CSV)


Notebook configured — will write to ./transactions.csv


In [11]:
# Helper functions
import math

def haversine(lat1, lon1, lat2, lon2):
    """Return distance in km between two lat/lon points (WGS84 approximation)."""
    R = 6371.0
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# Small utility to sample with replacement but reproducibly
def rchoice(seq):
    return seq[random.randint(0, len(seq)-1)]


In [12]:
# 1) Create customers and merchants — readable & commented
customers = []

# Urban centers used as seeds for realistic clustering
URBAN_CENTERS = [
    (28.7041, 77.1025, 1.0),   # Delhi
    (19.0760, 72.8777, 1.0),   # Mumbai
    (12.9716, 77.5946, 0.8),   # Bangalore
    (13.0827, 80.2707, 0.6),   # Chennai
    (22.5726, 88.3639, 0.5),   # Kolkata
    (26.9124, 75.7873, 0.3),   # Jaipur
]

center_weights = np.array([c[2] for c in URBAN_CENTERS], dtype=float)
center_weights /= center_weights.sum()

for i in range(1, N_CUSTOMERS + 1):
    cust_id = f'CUST_{i:05d}'
    # pick a city centroid biased by center_weights
    idx = np.random.choice(len(URBAN_CENTERS), p=center_weights)
    base_lat, base_lon = URBAN_CENTERS[idx][0], URBAN_CENTERS[idx][1]
    # jitter the home coordinates a bit — humans live near the city center but not exactly on it
    home_lat = base_lat + np.random.normal(scale=0.08)
    home_lon = base_lon + np.random.normal(scale=0.08)
    # simple spending profile: conservative / normal / high
    profile = np.random.choice(['conservative', 'normal', 'high'], p=[0.5, 0.4, 0.1])
    base_rate = {'conservative': 0.6, 'normal': 1.0, 'high': 2.0}[profile]
    # sticky category preferences (per-customer probability vector)
    cat_probs = np.random.dirichlet(np.ones(len(MERCHANT_CATEGORIES)))
    customers.append((cust_id, home_lat, home_lon, profile, base_rate, cat_probs.tolist()))

cust_df = pd.DataFrame(customers, columns=['customer_id', 'home_lat', 'home_long', 'profile', 'base_rate', 'cat_probs'])

# 2) Create merchants with realistic location jitter per category
merchants = []
category_base_probs = [0.25, 0.12, 0.12, 0.18, 0.18, 0.08, 0.07]
merchant_center_weights = center_weights  # reuse same city bias

# spread per category (higher for specialty stores)
sigma_by_cat = {
    'grocery': 0.02, 'gas': 0.02, 'restaurant': 0.03, 'retail': 0.04,
    'electronics': 0.06, 'jewelry': 0.06, 'luxury_goods': 0.08
}

for i in range(1, N_MERCHANTS + 1):
    mid = f'MERCHANT_{i:04d}'
    cat = np.random.choice(MERCHANT_CATEGORIES, p=category_base_probs)
    idx = np.random.choice(len(URBAN_CENTERS), p=merchant_center_weights)
    base_lat, base_lon = URBAN_CENTERS[idx][0], URBAN_CENTERS[idx][1]
    jitter = np.random.normal(scale=sigma_by_cat[cat])
    mlat = base_lat + (np.random.normal(scale=sigma_by_cat[cat]))
    mlon = base_lon + (np.random.normal(scale=sigma_by_cat[cat]))
    merchants.append((mid, cat, float(round(mlat,6)), float(round(mlon,6))))

merch_df = pd.DataFrame(merchants, columns=['merchant_id', 'merchant_category', 'merchant_lat', 'merchant_long'])

# Select a small set of collusive merchants (for merchant collusion scenario)
NUM_COLLUSIVE_MERCHANTS = 10
collusive_merchants = merch_df.sample(n=NUM_COLLUSIVE_MERCHANTS, random_state=SEED).merchant_id.tolist()

print('Created', len(cust_df), 'customers and', len(merch_df), 'merchants')


Created 5000 customers and 500 merchants


In [13]:
# 3) Generate baseline transactions (legitimate behavior)

# precompute total seconds in period for uniform sampling
total_seconds = int((END_DATE - START_DATE).total_seconds())

# hour seasonality (lunch + evening peaks)
hours = np.arange(24)
hour_probs = np.zeros(24)
# lunch peak (12-14)
hour_probs[12:15] += 0.18 / 3
# evening peak (18-21)
hour_probs[18:22] += 0.40 / 4
# morning small peak (8-10)
hour_probs[8:11] += 0.12 / 3
# spread remaining
hour_probs += (1.0 - hour_probs.sum()) / 24
hour_probs /= hour_probs.sum()

# day-of-week probabilities (weekend slightly busier for leisure)
dow_probs = np.array([0.12,0.14,0.14,0.14,0.14,0.16,0.16])
dow_probs /= dow_probs.sum()

# amount log-normal parameters per category (mu, sigma) on log scale
amt_params = {
    'grocery': (np.log(500), 0.6),
    'gas': (np.log(2000), 0.4),
    'restaurant': (np.log(1200), 0.7),
    'retail': (np.log(2500), 0.9),
    'electronics': (np.log(15000), 1.1),
    'jewelry': (np.log(35000), 1.2),
    'luxury_goods': (np.log(40000), 1.3)
}

# sample customers proportional to base_rate (high spenders more often)
cust_weights = cust_df['base_rate'].values
cust_weights = cust_weights / cust_weights.sum()
sampled_customer_indices = np.random.choice(np.arange(N_CUSTOMERS), size=N_ROWS, p=cust_weights)
sampled_customers = cust_df.iloc[sampled_customer_indices].reset_index(drop=True)

# quick mapping for card numbers (one primary card per customer)
card_numbers = [f'CARD_{100000 + i}' for i in range(1, N_CUSTOMERS+1)]
cust_card_map = dict(zip(cust_df.customer_id, card_numbers))

txn_rows = []
for i in range(N_ROWS):
    cust = sampled_customers.iloc[i]
    cid = cust['customer_id']
    # choose merchant category according to customer's preference
    cp = np.array(cust['cat_probs'])
    cp = cp / cp.sum()
    mcat = np.random.choice(MERCHANT_CATEGORIES, p=cp)
    # select a merchant id from that category (uniform among same-category merchants)
    mlist = merch_df[merch_df['merchant_category'] == mcat]['merchant_id'].tolist()
    if not mlist:
        merch_choice = merch_df.sample(n=1, random_state=SEED).merchant_id.values[0]
    else:
        merch_choice = random.choice(mlist)
    merch_info = merch_df[merch_df['merchant_id'] == merch_choice].iloc[0]
    mlat, mlon = merch_info['merchant_lat'], merch_info['merchant_long']

    # timestamp: pick a second in the period, then re-sample hour to respect hour_probs
    rand_seconds = random.randint(0, total_seconds)
    ts = START_DATE + timedelta(seconds=rand_seconds)
    sel_hour = np.random.choice(hours, p=hour_probs)
    ts = ts.replace(hour=int(sel_hour), minute=int(np.random.randint(0,60)), second=int(np.random.randint(0,60)))

    # amount: log-normal per category
    mu, sigma = amt_params[mcat]
    amount = float(np.exp(np.random.normal(mu, sigma)))
    amount = round(max(1.0, amount), 2)

    # compute distance from home to merchant
    distance_km = round(haversine(cust['home_lat'], cust['home_long'], mlat, mlon), 2)

    txn_rows.append({
        'customer_id': cid,
        'card_number': cust_card_map[cid],
        'timestamp': ts.strftime('%Y-%m-%dT%H:%M:%SZ'),
        'amount': amount,
        'merchant_id': merch_choice,
        'merchant_category': mcat,
        'merchant_lat': float(round(mlat,6)),
        'merchant_long': float(round(mlon,6)),
        'home_lat': float(round(cust['home_lat'],6)),
        'home_long': float(round(cust['home_long'],6)),
        'distance_from_home': distance_km
    })

base_df = pd.DataFrame(txn_rows)
# add transaction id and default fraud columns
base_df.insert(0, 'transaction_id', [f'TXN_{i:08d}' for i in range(1, len(base_df)+1)])
base_df['is_fraud'] = 0
base_df['fraud_type'] = 'none'
# convenience parsed ts
base_df['ts_dt'] = pd.to_datetime(base_df['timestamp'])
base_df['hour'] = base_df['ts_dt'].dt.hour
base_df['day_of_week'] = base_df['ts_dt'].dt.weekday
base_df['month'] = base_df['ts_dt'].dt.month

print('Generated baseline transactions:', base_df.shape)


Generated baseline transactions: (100000, 18)


In [14]:
# 4) Fraud pattern injections — refactored into functions for readability

def inject_merchant_collusion(df, merch_df, collusive_merchants, n_marked):
    """Force a subset of mule customers to transact at collusive merchants and mark some as fraud."""
    # pick mule customers (small set)
    mule_customers = df['customer_id'].drop_duplicates().sample(n=50, random_state=SEED).tolist()
    coll_txn_indices = df[df['customer_id'].isin(mule_customers)].sample(n=min(700, len(df)), random_state=SEED).index.tolist()

    for idx in coll_txn_indices:
        m = random.choice(collusive_merchants)
        df.at[idx, 'merchant_id'] = m
        coll_cat = random.choice(['jewelry', 'electronics', 'luxury_goods'])
        df.at[idx, 'merchant_category'] = coll_cat
        merch_row = merch_df[merch_df['merchant_id'] == m].iloc[0]
        df.at[idx, 'merchant_lat'] = merch_row['merchant_lat']
        df.at[idx, 'merchant_long'] = merch_row['merchant_long']
        # structured amounts around ₹49k or round numbers
        df.at[idx, 'amount'] = float(random.choice([49000, 49500, 49999, 5000, 10000, 25000]))
        df.at[idx, 'distance_from_home'] = round(haversine(df.at[idx,'home_lat'], df.at[idx,'home_long'], df.at[idx,'merchant_lat'], df.at[idx,'merchant_long']),2)

    # mark first n_marked of those coll_txn_indices as fraud
    for idx in coll_txn_indices[:n_marked]:
        df.at[idx, 'is_fraud'] = 1
        df.at[idx, 'fraud_type'] = 'merchant_collusion'
    return df


def inject_card_cloning(df, merch_df, n_to_inject):
    """Simulate impossible travel and concurrent sessions for some cards."""
    cloning_candidates = df.sample(n=2000, random_state=SEED+1).index.tolist()
    injected = 0
    for idx in cloning_candidates:
        if injected >= n_to_inject:
            break
        row = df.loc[idx]
        orig_ts = pd.to_datetime(row['timestamp'])
        lat1, lon1 = row['merchant_lat'], row['merchant_long']
        # find far merchants (>800 km)
        far_merch = merch_df.copy()
        far_merch['dist_from_t1'] = far_merch.apply(lambda r: haversine(lat1, lon1, r['merchant_lat'], r['merchant_long']), axis=1)
        far_candidates = far_merch[far_merch['dist_from_t1'] > 800]
        if far_candidates.empty:
            continue
        target = far_candidates.sample(n=1, random_state=SEED+injected).iloc[0]
        # create/modify another transaction to simulate same card used far away shortly after
        delta_minutes = random.randint(5, 120)
        new_ts = orig_ts + timedelta(minutes=delta_minutes)
        # pick an index to overwrite (simulates an additional transaction)
        other_idx = df.sample(n=1, random_state=SEED+100+injected).index[0]
        df.at[other_idx, 'customer_id'] = row['customer_id']
        df.at[other_idx, 'card_number'] = row['card_number']
        df.at[other_idx, 'merchant_id'] = target['merchant_id']
        df.at[other_idx, 'merchant_category'] = target['merchant_category']
        df.at[other_idx, 'merchant_lat'] = target['merchant_lat']
        df.at[other_idx, 'merchant_long'] = target['merchant_long']
        df.at[other_idx, 'timestamp'] = new_ts.strftime('%Y-%m-%dT%H:%M:%SZ')
        df.at[other_idx, 'ts_dt'] = pd.to_datetime(df.at[other_idx, 'timestamp'])
        df.at[other_idx, 'hour'] = df.at[other_idx,'ts_dt'].hour
        df.at[other_idx, 'day_of_week'] = df.at[other_idx,'ts_dt'].weekday()
        df.at[other_idx, 'month'] = df.at[other_idx,'ts_dt'].month
        df.at[other_idx, 'amount'] = round(max(50.0, float(np.exp(np.random.normal(*amt_params[target['merchant_category']])))),2)
        df.at[other_idx, 'distance_from_home'] = round(haversine(df.at[other_idx,'home_lat'], df.at[other_idx,'home_long'], df.at[other_idx,'merchant_lat'], df.at[other_idx,'merchant_long']),2)
        df.at[other_idx, 'is_fraud'] = 1
        df.at[other_idx, 'fraud_type'] = 'card_cloning'
        injected += 1
    return df


def inject_account_takeover(df, merch_df, n_to_inject):
    """Simulate bursts of high-value transactions for some customers (ATO patterns)."""
    ato_customers = df['customer_id'].drop_duplicates().sample(n=int(0.01 * N_CUSTOMERS), random_state=SEED+2).tolist()
    injected = 0
    for cust in ato_customers:
        if injected >= n_to_inject:
            break
        cust_idxs = df[df['customer_id'] == cust].index.tolist()
        if len(cust_idxs) < 3:
            continue
        k = random.randint(3, 8)
        chosen = random.sample(cust_idxs, min(k, len(cust_idxs)))
        # select a compromise time
        compromise_time = (START_DATE + timedelta(seconds=random.randint(0, total_seconds))).replace(tzinfo=timezone.utc)
        base_increase = random.uniform(3.0, 10.0)
        for j, idx in enumerate(chosen):
            new_ts = compromise_time + timedelta(minutes=j * random.randint(1,10))
            df.at[idx, 'timestamp'] = new_ts.strftime('%Y-%m-%dT%H:%M:%SZ')
            df.at[idx, 'ts_dt'] = pd.to_datetime(df.at[idx,'timestamp'])
            df.at[idx, 'hour'] = df.at[idx,'ts_dt'].hour
            df.at[idx, 'day_of_week'] = df.at[idx,'ts_dt'].weekday()
            df.at[idx, 'month'] = df.at[idx,'ts_dt'].month
            old_amt = df.at[idx,'amount']
            new_amt = round(old_amt * base_increase * (1 + np.random.normal(0, 0.2)), 2)
            df.at[idx,'amount'] = float(max(100.0, min(new_amt, 150000.0)))
            # sometimes switch to high-value categories
            if random.random() < 0.7:
                new_cat = random.choice(['electronics','jewelry','luxury_goods'])
                mlist = merch_df[merch_df['merchant_category'] == new_cat]['merchant_id'].tolist()
                if mlist:
                    mid = random.choice(mlist)
                    merch_row = merch_df[merch_df['merchant_id'] == mid].iloc[0]
                    df.at[idx,'merchant_id'] = mid
                    df.at[idx,'merchant_category'] = new_cat
                    df.at[idx,'merchant_lat'] = merch_row['merchant_lat']
                    df.at[idx,'merchant_long'] = merch_row['merchant_long']
                    df.at[idx,'distance_from_home'] = round(haversine(df.at[idx,'home_lat'], df.at[idx,'home_long'], df.at[idx,'merchant_lat'], df.at[idx,'merchant_long']),2)
            df.at[idx,'is_fraud'] = 1
            df.at[idx,'fraud_type'] = 'account_takeover'
            injected += 1
            if injected >= n_to_inject:
                break
    return df


def inject_velocity_probes(df, max_inject=200):
    """Small-value rapid-fire probes used to test cards; some portion marked fraud."""
    probe_cards = df['card_number'].drop_duplicates().sample(n=100, random_state=SEED+3).tolist()
    probe_injected = 0
    for card in probe_cards:
        if probe_injected >= max_inject:
            break
        idxs = df[df['card_number'] == card].sample(n=min(10, df[df['card_number']==card].shape[0]), random_state=SEED+4).index.tolist()
        base_time = (START_DATE + timedelta(seconds=random.randint(0, total_seconds))).replace(tzinfo=timezone.utc)
        for j, idx in enumerate(idxs):
            new_ts = base_time + timedelta(seconds=j * random.randint(5, 60))
            df.at[idx,'timestamp'] = new_ts.strftime('%Y-%m-%dT%H:%M:%SZ')
            df.at[idx,'ts_dt'] = pd.to_datetime(df.at[idx,'timestamp'])
            df.at[idx,'hour'] = df.at[idx,'ts_dt'].hour
            df.at[idx,'day_of_week'] = df.at[idx,'ts_dt'].weekday()
            df.at[idx,'month'] = df.at[idx,'ts_dt'].month
            df.at[idx,'amount'] = float(round(np.random.choice([10,20,50,99,199,499]) * (1 + np.random.normal(0,0.05)),2))
            if random.random() < 0.3 and df.at[idx,'is_fraud'] == 0:
                df.at[idx,'is_fraud'] = 1
                df.at[idx,'fraud_type'] = 'card_cloning'
                probe_injected += 1
    return df

# Now call the injectors in sequence with tuned counts

df = base_df.copy()

# Pre-calculate target counts
TARGET_FRAUD_COUNT = int(N_ROWS * TARGET_FRAUD_RATE)
print('Target fraud count:', TARGET_FRAUD_COUNT)

# allocate proportions
pct_cloning = 0.4
pct_ato = 0.4
pct_collusion = 0.2
n_cloning = int(TARGET_FRAUD_COUNT * pct_cloning)
n_ato = int(TARGET_FRAUD_COUNT * pct_ato)
n_collusion = TARGET_FRAUD_COUNT - n_cloning - n_ato

# inject
print('Injecting merchant collusion...')
df = inject_merchant_collusion(df, merch_df, collusive_merchants, n_marked=n_collusion)
print('Injecting card cloning...')
df = inject_card_cloning(df, merch_df, n_to_inject=n_cloning)
print('Injecting account takeover events...')
df = inject_account_takeover(df, merch_df, n_to_inject=n_ato)
print('Injecting velocity probes...')
df = inject_velocity_probes(df, max_inject=200)

# Some legitimate holiday spikes (not fraud) to reduce false positives
holiday_customers = df['customer_id'].drop_duplicates().sample(n=200, random_state=SEED+5).tolist()
for cust in holiday_customers:
    idxs = df[df['customer_id'] == cust].sample(n=min(3, df[df['customer_id']==cust].shape[0]), random_state=SEED+6).index.tolist()
    for idx in idxs:
        df.at[idx,'amount'] = round(df.at[idx,'amount'] * random.uniform(2.0, 5.0),2)
        df.at[idx,'is_fraud'] = 0
        df.at[idx,'fraud_type'] = 'none'

print('Fraud injection completed — current frauds:', int(df['is_fraud'].sum()))


Target fraud count: 2000
Injecting merchant collusion...
Injecting card cloning...
Injecting account takeover events...
Injecting velocity probes...
Fraud injection completed — current frauds: 1656


In [15]:
# 5) Risk-score post-processing + ensure global fraud rate ~ TARGET_FRAUD_RATE

# compute initial simple risk signals
# These are short, human-friendly heuristics used to bias final labeling

def compute_risk_score_row(r):
    score = 0.0
    # large distance
    if r['distance_from_home'] > 200:
        score += 2.0
    # very large amounts
    if r['amount'] > 50000:
        score += 1.5
    # odd hours (night)
    if int(r['hour']) in range(0,6):
        score += 0.8
    # flagged as one of the injected frauds already
    if r['fraud_type'] in ['merchant_collusion','card_cloning','account_takeover']:
        score += 2.5
    # round-number structuring
    if int(r['amount']) % 1000 == 0:
        score += 0.6
    return score

# compute risk_score for all rows
risk_scores = df.apply(compute_risk_score_row, axis=1)
# convert to a probability with a logistic transform; threshold will be tuned
threshold = 2.5
p_fraud = 1 / (1 + np.exp(-(risk_scores - threshold)))

# sample probabilistically by these p_fraud values but keep previously injected frauds as high-probability
rng = np.random.RandomState(SEED+10)
candidate_flags = (rng.rand(len(df)) < p_fraud).astype(int)
# combine with existing flags to form a tentative is_fraud
# we keep explicit injected frauds where df['is_fraud'] == 1
final_flags = ((df['is_fraud'] == 1) | (candidate_flags == 1)).astype(int)

current_count = int(final_flags.sum())
print('Tentative fraud count after risk-sampling:', current_count)

# If we overshot / undershot, adjust by ranking risk_scores
TARGET_COUNT = int(TARGET_FRAUD_RATE * len(df))
if current_count != TARGET_COUNT:
    print('Adjusting to hit exact target fraud count:', TARGET_COUNT)
    # Rank rows by risk_scores descending
    rank_order = np.argsort(-risk_scores.values)
    # build new final_flags all zeros then set top TARGET_COUNT by rank to 1
    new_flags = np.zeros(len(df), dtype=int)
    top_idxs = rank_order[:TARGET_COUNT]
    new_flags[top_idxs] = 1
    final_flags = pd.Series(new_flags, index=df.index)

# apply final flags and set fraud_type for newly marked cases (if previously none, set to 'synthetic_rule')
df['is_fraud'] = final_flags.astype(int)
df.loc[(df['is_fraud'] == 1) & (df['fraud_type'] == 'none'), 'fraud_type'] = 'synthetic_rule'

print('Final fraud count:', int(df['is_fraud'].sum()), 'target:', TARGET_COUNT)

# final housekeeping: recompute distance and timestamp formats
from datetime import timezone

df['distance_from_home'] = df.apply(lambda r: round(haversine(r['home_lat'], r['home_long'], r['merchant_lat'], r['merchant_long']),2), axis=1)
# ensure transaction ids are contiguous
df['transaction_id'] = [f'TXN_{i:08d}' for i in range(1, len(df)+1)]
# iso timestamp
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

final_cols = [
    'transaction_id', 'customer_id', 'card_number', 'timestamp', 'amount',
    'merchant_id', 'merchant_category', 'merchant_lat', 'merchant_long',
    'is_fraud', 'fraud_type', 'hour', 'day_of_week', 'month', 'distance_from_home'
]
out_df = df[final_cols].copy()

# sanity checks
assert len(out_df) == N_ROWS
print('Sanity OK — rows:', len(out_df))
print('Fraud fraction:', out_df['is_fraud'].mean())

# write CSV to disk
os.makedirs(OUT_DIR, exist_ok=True)
out_df.to_csv(OUT_CSV, index=False)
print('Wrote CSV to', OUT_CSV)


Tentative fraud count after risk-sampling: 38505
Adjusting to hit exact target fraud count: 2000
Final fraud count: 2000 target: 2000
Sanity OK — rows: 100000
Fraud fraction: 0.02
Wrote CSV to ./transactions.csv
